<a href="https://colab.research.google.com/github/tmtong/ai-library-exploration/blob/main/autogluon_tabular.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
!pip install -U pip
!pip install -U setuptools wheel
!pip install -U "mxnet<2.0.0"
!pip install autogluon

In [17]:
!pip install git+https://github.com/fastai/fastai.git
from fastai.tabular.all import *
path = untar_data(URLs.ADULT_SAMPLE)
path.ls()

  Cloning https://github.com/fastai/fastai.git to /tmp/pip-req-build-ii5l_zxw
  Running command git clone -q https://github.com/fastai/fastai.git /tmp/pip-req-build-ii5l_zxw


(#3) [Path('/root/.fastai/data/adult_sample/adult.csv'),Path('/root/.fastai/data/adult_sample/export.pkl'),Path('/root/.fastai/data/adult_sample/models')]

In [18]:
from autogluon.tabular import TabularDataset, TabularPredictor
import pandas as pd
datapath = path/'adult.csv'
df = pd.read_csv(datapath)

msk = np.random.rand(len(df)) < 0.8
train_df = df[msk]
test_df = df[~msk]


train_df = TabularDataset(train_df)
train_df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,49,Private,101320,Assoc-acdm,12.0,Married-civ-spouse,NaN,Wife,White,Female,0,1902,40,United-States,>=50k
1,44,Private,236746,Masters,14.0,Divorced,Exec-managerial,Not-in-family,White,Male,10520,0,45,United-States,>=50k
2,38,Private,96185,HS-grad,NaN,Divorced,NaN,Unmarried,Black,Female,0,0,32,United-States,<50k
3,38,Self-emp-inc,112847,Prof-school,15.0,Married-civ-spouse,Prof-specialty,Husband,Asian-Pac-Islander,Male,0,0,40,United-States,>=50k
4,42,Self-emp-not-inc,82297,7th-8th,NaN,Married-civ-spouse,Other-service,Wife,Black,Female,0,0,50,United-States,<50k


In [19]:
label = 'salary'
predictor = TabularPredictor(label=label).fit(train_df)

No path specified. Models will be saved in: "AutogluonModels/ag-20210710_023018/"
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels/ag-20210710_023018/"
AutoGluon Version:  0.2.0
Train Data Rows:    26072
Train Data Columns: 14
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  ['>=50k', '<50k']
	If 'binary' is not the correct problem_type, please manually specify the problem_type argument in fit() (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> label mapping:  class 1 = >=50k, class 0 = <50k
	Note: For your binary classification, AutoGluon arbitrarily selected which label-value represents positive (>=50k) vs negative (<50k) class.
	To explicitly set the positive_class, either rename classes to 1 and 0, or specify positive_class in Predictor init.
Using Feature Generators to preprocess the data ...
Fitting

In [20]:
y_test = test_df[label]  # values to predict
test_data_nolab = test_df.drop(columns=[label])  # delete label column to prove we're not cheating
test_data_nolab.head()
y_pred = predictor.predict(test_data_nolab)
print("Predictions:  \n", y_pred)
perf = predictor.evaluate_predictions(y_true=y_test, y_pred=y_pred, auxiliary_metrics=True)

Evaluation: accuracy on test data: 0.8770226537216829
Evaluations on test data:
{
    "accuracy": 0.8770226537216829,
    "balanced_accuracy": 0.8001225884286338,
    "mcc": 0.6449887313614096,
    "f1": 0.7174220963172805,
    "precision": 0.7963836477987422,
    "recall": 0.6527061855670103
}


Predictions:  
 7         <50k
8         <50k
13        <50k
17        <50k
19       >=50k
         ...  
32546    >=50k
32548     <50k
32552     <50k
32556    >=50k
32560     <50k
Name: salary, Length: 6489, dtype: object
